## 휴먼지능정보공학과 201810808 정민지
### 13주 과제
블록체인에서 주문을 하고, 그 내역을 로컬 파일에 작성해 보자.

블록체인으로 주문을 전송하면, 이벤트가 발생하고 주문내역을 로컬파일에 쓰게 된다.

* EventTest.sol에서는 이벤트가 발생하면서 그 로그를 파일에 작성한다 (OrderEvent.txt)

* OrderEvent.sol을 수정하여:

	* order(상품항목, 주문 개수, 배송지) 함수 작성

	* getBalance() 함수 작성. 해당 프로그램 생성한 Owner만 읽을 수 있게 modfier isOnwer 작성.



geth 또는 ganache 어느 하나를 선택하여 프로그램을 작성하세요.

아래 로그가 파일에 쓰여지도록 하고, 노트북에서 그 파일을 출력하세요 (!type src/OrderEvent.txt)

전송자주소, 1111, 3, 20 2-gil Hongji-dong Jongro-gu Seoul

전송자주소, 1111, 5, 20 2-gil Hongji-dong Jongro-gu Seoul

전송자주소, 1111, 20, 20 2-gil Hongji-dong Jongro-gu Seoul

In [15]:
%%writefile src/OrderEvent.sol
pragma solidity 0.5.1;

contract OrderEvent {
    
    uint unitPrice = 10;
    address payable owner;
    //event OrderLog(bytes2 _itemId, uint indexed _value);
    event OrderLog(address indexed _from, bytes2 _itemId, uint indexed quantity, string addr);
    
    constructor() public {
        owner = msg.sender;
    }
    
    function order(bytes2 _itemId, uint quantity,string memory addr) public payable{
        if (msg.sender == owner) {
            uint256 orderAmount = quantity * unitPrice;
            require(msg.value == orderAmount);
            //emit OrderLog(_itemId, msg.value);
            emit OrderLog(msg.sender, _itemId, quantity,addr);
        }
    }
    
    modifier isOwner() { 
        if (msg.sender != owner) {
            revert();
        }
        _; //continue executing rest of method body
    }
    
    function getBalance() view public isOwner returns(uint) {
        return address(this).balance;
    }
}

Overwriting src/OrderEvent.sol


In [16]:
!solc --abi --bin --gas src/OrderEvent.sol


======= src/OrderEvent.sol:OrderEvent =======
Gas estimation:
construction:
   40492 + 178800 = 219292
external:
   getBalance():	877
   order(bytes2,uint256,string):	infinite
Binary: 
6080604052600a60005534801561001557600080fd5b5033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555061037e806100666000396000f3fe608060405260043610610046576000357c01000000000000000000000000000000000000000000000000000000009004806312065fe01461004b5780637a59a92914610076575b600080fd5b34801561005757600080fd5b50610060610166565b6040518082815260200191505060405180910390f35b6101646004803603606081101561008c57600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916906020019092919080359060200190929190803590602001906401000000008111156100de57600080fd5b8201836020820111156100f057600080fd5b8035906020019184600183028401116401000000008311171561011257600080fd5b91908080601f016020809104026020016040519081016040528093929190

In [17]:
%%writefile src/OrderEventDeploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray =[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_itemId","type":"bytes2"},{"name":"quantity","type":"uint256"},{"name":"addr","type":"string"}],"name":"order","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_from","type":"address"},{"indexed":false,"name":"_itemId","type":"bytes2"},{"indexed":true,"name":"quantity","type":"uint256"},{"indexed":false,"name":"addr","type":"string"}],"name":"OrderLog","type":"event"}];
var _bin="0x" + "6080604052600a60005534801561001557600080fd5b5033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555061037e806100666000396000f3fe608060405260043610610046576000357c01000000000000000000000000000000000000000000000000000000009004806312065fe01461004b5780637a59a92914610076575b600080fd5b34801561005757600080fd5b50610060610166565b6040518082815260200191505060405180910390f35b6101646004803603606081101561008c57600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916906020019092919080359060200190929190803590602001906401000000008111156100de57600080fd5b8201836020820111156100f057600080fd5b8035906020019184600183028401116401000000008311171561011257600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f8201169050808301925050505050505091929192905050506101e1565b005b6000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161415156101c457600080fd5b3073ffffffffffffffffffffffffffffffffffffffff1631905090565b600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16141561034d576000805483029050803414151561024d57600080fd5b823373ffffffffffffffffffffffffffffffffffffffff167f0cee52e22e6b387c4183ee502522fa33f030e118cc6065c7a842e8b7c00e4a6a868560405180837dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff19167dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916815260200180602001828103825283818151815260200191508051906020019080838360005b838110156103105780820151818401526020810190506102f5565b50505050905090810190601f16801561033d5780820380516001836020036101000a031916815260200191505b50935050505060405180910390a3505b50505056fea165627a7a72305820560da9061da1af60ca953dbace7d2a064b565d83b411caec5775810088dfa71c0029";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 2000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })

    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/OrderEventDeploy.js


In [18]:
!node src/OrderEventDeploy.js

Deploying the contract from 0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486
hash: 0x6b1c23c0fe39f834c688ceece11cf1579cffacb0e841dc008cb9abe248b5c313
---> The contract deployed to: 0x9a48Cd64348A4Fe938BFF03E40c28F42d955329e


In [5]:
!geth --exec "eth.getTransactionReceipt('0x60403fc2ad150315e49e087d9bd64f7715b56e43bc8684aa60285e428d0055ea')" attach http://localhost:8445

{
  blockHash: "0x613240f178ac0dc240c497dfa63ee67544c11dd34a93b40bbbfd59e6220495d0",
  blockNumber: 688,
  contractAddress: "0x6a4cfc72fe3c88d9f2a1d0b22cf7c75a32eaa6d2",
  cumulativeGasUsed: 335348,
  from: "0x8e71085ffefccb88a4b54d4c93a866db7928a486",
  gasUsed: 335348,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  root: "0xc2e03b7762b25cdd93e22a2e926998dd34987e6da09c93f5134c72c4ff4a313c",
  to: null,
  transactionHash: "0x60403fc2ad150315e49e087d9bd64f7715b56e43bc8684aa60285e428d0055ea",
  tran

In [20]:
%%writefile src/OrderEventUse.js
var Web3=require('web3');
var fs=require('fs');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8446"));
}
var _abiArray =[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_itemId","type":"bytes2"},{"name":"quantity","type":"uint256"},{"name":"addr","type":"string"}],"name":"order","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_from","type":"address"},{"indexed":false,"name":"_itemId","type":"bytes2"},{"indexed":true,"name":"quantity","type":"uint256"},{"indexed":false,"name":"addr","type":"string"}],"name":"OrderLog","type":"event"}];

async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, '0x9a48Cd64348A4Fe938BFF03E40c28F42d955329e');
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    var event = _order.events.OrderLog({
            filter: {_from: accounts[0]},
            //fromBlock: 'latest', toBlock: 'pending'
        }, function (error, result) {
        if (!error) {
            var log = result.returnValues;
            console.log("Event fired: " + JSON.stringify(log));
            //console.log(Object.keys(log));
            fs.appendFile("src/OrderEvent.txt",log["_from"]+", "+log["_itemId"]+","+log["quantity"]+', '+log["addr"]+"\n" , "utf-8", function(e){
            if(!e) {
                console.log(">> Writing to file");
                
                }
            });
        }
    });
    
    var value;
    my = await _order.methods.order("0x1111", 3, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000,value:30})
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    my = await _order.methods.order("0x1111", 5, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000,value:50})
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    my = await _order.methods.order("0x1111", 20, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000,value:200})
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    process.exit(1);
    
}

doIt();


Overwriting src/OrderEventUse.js


In [21]:
!node src/OrderEventUse.js

Account: 0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486
Event fired: {"0":"0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486","1":"0x1111","2":"3","3":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486","_itemId":"0x1111","quantity":"3","addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}
---> MyFunction called {"0":"0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486","1":"0x1111","2":"3","3":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486","_itemId":"0x1111","quantity":"3","addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}
>> Writing to file
Event fired: {"0":"0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486","1":"0x1111","2":"5","3":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486","_itemId":"0x1111","quantity":"5","addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}
>> Writing to file
---> MyFunction called {"0":"0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486","1":"0x1111","2":"5","3":"20 2-gil Hongji-do

In [22]:
!type src\OrderEvent.txt

0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486, 0x1111,3, 20 2-gil Hongji-dong Jongro-gu Seoul
0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486, 0x1111,5, 20 2-gil Hongji-dong Jongro-gu Seoul
0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486, 0x1111,20, 20 2-gil Hongji-dong Jongro-gu Seoul
